In [0]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [0]:
import os
from tqdm import tqdm
import cv2
import csv
import ast
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
#from skimage.morphology import skeletonize, thin

from keras import layers
from keras import models
from keras import optimizers
#from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import img_to_array, load_img

In [0]:
folders = os.listdir('/content/drive/My Drive/datasets/SigResize/Genuine')
folders.sort(key=lambda x: int(x))

In [0]:
genuine_path = '/content/drive/My Drive/datasets/SigResize/Genuine'
forgery_path = '/content/drive/My Drive/datasets/SigResize/Forgery/Opposite Hand'
#forgery_path = '/content/drive/My Drive/datasets/SigResize/Forgery/Skilled'
randomcsv_path = '/content/drive/My Drive/datasets/SigResize/randomcsv'
modelsave_path = '/content/drive/My Drive/datasets/SigResize/modelsaveS22Imgss'
modelsave12_path = '/content/drive/My Drive/datasets/SigResize/Model12'
result_path = '/content/drive/My Drive/datasets/SigResize/result'
preprocessed_path = '/content/drive/My Drive/datasets/SigResize/preprocessed'

In [0]:
def segmentImage(image):  
  hHist=np.zeros(shape=image.shape[0], dtype=int)
  vHist=np.zeros(shape=image.shape[1], dtype=int)

  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if(image[i][j]==0):
        hHist[i]+=1
        vHist[j]+=1
  
  locLeft=0
  locRight=image.shape[0]
  locTop=0
  locBottom=image.shape[1]
  
  count=0
  for i in range(hHist.shape[0]):
    if(count<=0):
        count=0
        if(hHist[i]!=0):
            locTop=i
            count+=1
    else:
        if(hHist[i]!=0):
            count+=1
        else:
            count-=hHist.shape[0]/100

        if(count>hHist.shape[0]/30):
            break
            
  count=0
  for i in reversed(range(hHist.shape[0])):
    if(count<=0):
        count=0
        if(hHist[i]!=0):
            locBottom=i
            count+=1
    else:
        if(hHist[i]!=0):
            count+=1
        else:
            count-=hHist.shape[0]/100

        if(count>hHist.shape[0]/30):
            break
            
  count=0
  for i in range(vHist.shape[0]):
    if(count<=0):
        count=0
        if(vHist[i]!=0):
            locLeft=i
            count+=1
    else:
        if(vHist[i]!=0):
            count+=1
        else:
            count-=vHist.shape[0]/100

        if(count>vHist.shape[0]/30):
            break
            
  count=0
  for i in reversed(range(vHist.shape[0])):
    if(count<=0):
        count=0
        if(vHist[i]!=0):
            locRight=i
            count+=1
    else:
        if(vHist[i]!=0):
            count+=1
        else:
            count-=vHist.shape[0]/100

        if(count>vHist.shape[0]/30):
            break
            
  return locLeft, locRight, locTop, locBottom
			
def preProcessImageAll(train_path, final_img_size = (300,300), power_law=False, segment=True, log_transform=False):
  train_batch = os.listdir(train_path)
  x_train = []
  train_data = [x for x in train_batch if x.endswith('png') or x.endswith('PNG') or x.endswith('jpg') or x.endswith('JPG') or x.endswith('tif') or x.endswith('TIF')]
  train_data.sort(key = lambda x: int(x.split('.')[0]))

  for sample in tqdm(train_data):
    img_path = os.path.join(train_path, sample)
    #importing images from drive
    #x = image.load_img(img_path)
    #img = image.img_to_array(x)
    img = cv2.imread(img_path)
        
    #Perfom Median blur on image
    mbvalue = int(np.max(img.shape)/200)
    mbvalue = mbvalue if mbvalue%2==1 else mbvalue+1
    img = cv2.medianBlur(img, mbvalue)

    #changing RGB to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    #resize image to 600xH
    img = cv2.resize(img, (600, int(600*float(img.shape[0])/img.shape[1])))
    
    #if power_law enabled
    if(power_law):
      img = img**0.9
      img[img>255]=255
      img[img<0]=0
      img = img.astype('uint8')
          
    #denoising the grayscale image
    img = cv2.fastNlMeansDenoising(img, None, 10, 21)
    
    if (log_transform):
        img = (np.log(img+1)/(np.log(10+np.max(img))))*255
        img=img.astype('uint8')
    
    #Threshold binary image
    avg = np.average(img)
    _,image = cv2.threshold(img, int(avg)-30, 255, cv2.THRESH_BINARY)
            
    #segment the signature section only
    if(segment):
      seg = segmentImage(image)
      image = image[seg[2]:seg[3], seg[0]:seg[1]]
          
    #padding to make image into square
    lp, rp, tp, bp = (0,0,0,0)
    if(image.shape[0]>image.shape[1]):
      lp = int((image.shape[0]-image.shape[1])/2)
      rp = lp
    elif(image.shape[1]>image.shape[0]):
      tp = int((image.shape[1]-image.shape[0])/2)
      bp = tp
    image_padded = cv2.copyMakeBorder(image, tp, bp, lp, rp, cv2.BORDER_CONSTANT, value=255)

    #resizing the image
    img = cv2.resize(image_padded, final_img_size)

    #producing image negative
    img = 255-img

    #skeletonizing image
    #img = thin(img/255)

    img = img.astype('bool')

    #appending it in list
    x_train.append(img)

  #converting it into np-array  
  x_train = np.array(x_train)
  return x_train

def preProcessImage(train_path, final_img_size = (300,300), length=0, getEnd=False, power_law=False, segment=True, log_transform=False):
  train_batch = os.listdir(train_path)
  x_train = []
  train_data = [x for x in train_batch if x.endswith('png') or x.endswith('PNG') or x.endswith('jpg') or x.endswith('JPG') or x.endswith('tif') or x.endswith('TIF')]
  train_data.sort(key = lambda x: int(x.split('.')[0]))
  
  if(length==0 or (length>len(train_data))):
    length = len(train_data)

  for sample in tqdm(train_data[:length]):
    img_path = os.path.join(train_path, sample)
    #importing images from drive
    #x = image.load_img(img_path)
    #img = image.img_to_array(x)
    img = cv2.imread(img_path)
        
    #Perfom Median blur on image
    mbvalue = int(np.max(img.shape)/200)
    mbvalue = mbvalue if mbvalue%2==1 else mbvalue+1
    img = cv2.medianBlur(img, mbvalue)

    #changing RGB to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    #resize image to 600xH
    img = cv2.resize(img, (600, int(600*float(img.shape[0])/img.shape[1])))
    
    #if power_law enabled
    if(power_law):
      img = img**0.9
      img[img>255]=255
      img[img<0]=0
      img = img.astype('uint8')
          
    #denoising the grayscale image
    img = cv2.fastNlMeansDenoising(img, None, 10, 21)
    
    if (log_transform):
        img = (np.log(img+1)/(np.log(10+np.max(img))))*255
        img=img.astype('uint8')
    
    #Threshold binary image
    avg = np.average(img)
    _,image = cv2.threshold(img, int(avg)-30, 255, cv2.THRESH_BINARY)
            
    #segment the signature section only
    if(segment):
      seg = segmentImage(image)
      image = image[seg[2]:seg[3], seg[0]:seg[1]]
          
    #padding to make image into square
    lp, rp, tp, bp = (0,0,0,0)
    if(image.shape[0]>image.shape[1]):
      lp = int((image.shape[0]-image.shape[1])/2)
      rp = lp
    elif(image.shape[1]>image.shape[0]):
      tp = int((image.shape[1]-image.shape[0])/2)
      bp = tp
    image_padded = cv2.copyMakeBorder(image, tp, bp, lp, rp, cv2.BORDER_CONSTANT, value=255)

    #resizing the image
    img = cv2.resize(image_padded, final_img_size)

    #producing image negative
    img = 255-img

    #skeletonizing image
    #img = thin(img/255)

    img = img.astype('bool')

    #appending it in list
    x_train.append(img)

  #converting it into np-array  
  x_train = np.array(x_train)
  return x_train[:length-5], x_train[-5:]

def convertToInt(arr):
  t1=[]
  for x in arr:
    t2=[]
    for y in x:
      t3=[]
      for z in y:
        if(z==True):
          t3.append(1)
        else:
          t3.append(0)
      t2.append(np.array(t3))
    t1.append(np.array(t2))
  return np.array(t1).astype('uint8')

def convertToBool(arr):
  t1=[]
  for x in arr:
    t2=[]
    for y in x:
      t3=[]
      for z in y:
        if(z==1):
          t3.append(True)
        else:
          t3.append(False)
      t2.append(np.array(t3))
    t1.append(np.array(t2))
  return np.array(t1)


def csvWriter(fil_name, nparray):
  example = nparray.tolist()
  with open(fil_name+'.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(example)


def csvReader(fil_name):
  with open(fil_name+'.csv', 'r') as f:
    reader = csv.reader(f)
    examples = list(reader)
    examples = np.array(examples)
  
  t1=[]
  for x in examples:
    t2=[]
    for y in x:
      z= ast.literal_eval(y)
      t2.append(np.array(z))
    t1.append(np.array(t2))
  ex = np.array(t1)
  return ex

In [0]:
x_random = csvReader(os.path.join(randomcsv_path,'random3'))

In [0]:
w=False

f=open(os.path.join(result_path, 'S22Imgresult.txt'), 'a')
g=open(os.path.join(result_path, 'S22Imggen.txt'), 'a')
fo = open(os.path.join(result_path, 'S22Imgfor.txt'),'a')
if w:
  f.write("LENGTH = Train(22) Test(5)")

for fol in folders[0:10]:
  print("Train "+fol)
  mod = models.Sequential()
  mod.add(layers.Conv2D(16, (9,9), activation='relu', input_shape=(300,300,1)))
  mod.add(layers.MaxPooling2D((2,2)))
  mod.add(layers.Conv2D(32, (5,5), activation='relu'))
  mod.add(layers.MaxPooling2D((2,2)))
  mod.add(layers.Conv2D(32, (3,3), activation='relu'))
  mod.add(layers.MaxPooling2D((3,3)))
  mod.add(layers.Conv2D(16, (2,2), activation='relu'))
  mod.add(layers.MaxPooling2D((2,2)))
  mod.add(layers.Flatten())
  mod.add(layers.Dropout(0.2))
  mod.add(layers.Dense(256, activation='relu'))
  mod.add(layers.Dropout(0.4))
  mod.add(layers.Dense(128, activation='relu'))
  mod.add(layers.Dense(1, activation='sigmoid'))
    
  mod.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-3), metrics=['acc'])
  
  #mod.summary()
  
  #if(os.path.isdir(os.path.join(genuine_path, fol)) and os.path.isdir(os.path.join(forgery_path, fol))):
  if(True):
#     x_genuine, x_test = preProcessImage(os.path.join(genuine_path, fol), length=14)
#     x_forgery = preProcessImageAll(os.path.join(forgery_path, fol))
    x_img = csvReader(os.path.join(preprocessed_path, fol, fol))
    x_genuine = x_img[:22]
    x_test = x_img[22:]
    x_forgery = csvReader(os.path.join(preprocessed_path, fol, fol+'Forgery'))
    if(os.path.isfile(os.path.join(modelsave_path, fol, fol+'.h5'))):
      mod.load_weights(os.path.join(modelsave_path, fol, fol+'.h5'))
    else:
      mod.fit(np.concatenate((x_genuine, x_random)).reshape(x_genuine.shape[0]+x_random.shape[0], 300,300,1), np.concatenate((np.full(x_genuine.shape[0],1),np.full(x_random.shape[0],0))), epochs=20, verbose=1, shuffle=True)
      if(os.path.isdir(os.path.join(modelsave_path, fol))==False):
        os.mkdir(os.path.join(modelsave_path, fol))
      mod.save_weights(os.path.join(modelsave_path, fol, fol+'.h5'))
      
    print('\n'+fol+'\nGenuine: ')
    if w:
      f.write('\n'+fol+'\nGenuine: ')
    for i in x_test:
      predict = mod.predict(np.array([i.reshape(300,300,1),]))
      print('+ ', str(predict*100), '%', end='   ')
      if w:
        f.write('+ '+ str(predict[0][0]*100)+ '%, ')
        g.write(str(predict[0][0]*100)+', ')
    
    print('\nForgery: ')
    if w:
      f.write('\nForgery: ')
    for i in x_forgery:
      predict = mod.predict(np.array([i.reshape(300,300,1),]))
      print('- ', str(predict*100), '%', end='   ')
      if w:
        f.write('- '+ str(predict[0][0]*100)+ '%, ')
        fo.write(str(predict[0][0]*100)+ ', ')
    print('\n')
    if w:
      f.write('\n')
    
f.close()  
g.close()
fo.close()


Train 1
Epoch 1/20
137/137 [==============================] - 15s 107ms/step - loss: 0.5415 - acc: 0.8394
Epoch 2/20
137/137 [==============================] - 0s 2ms/step - loss: 0.3549 - acc: 0.8394
Epoch 3/20
137/137 [==============================] - 0s 2ms/step - loss: 0.2299 - acc: 0.8540
Epoch 4/20
137/137 [==============================] - 0s 2ms/step - loss: 0.1145 - acc: 0.9635
Epoch 5/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0626 - acc: 0.9781
Epoch 6/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0338 - acc: 0.9781
Epoch 7/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0033 - acc: 1.0000
Epoch 8/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0045 - acc: 1.0000
Epoch 9/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0148 - acc: 0.9927
Epoch 10/20
137/137 [==============================] - 0s 2ms/step - loss: 0.0061 - acc: 1.0000
Epoch 11/20
137/137 [=================

**Done**

In [0]:
w=True
f=open(os.path.join(result_path, 'Nresult.txt'), 'a')
g = open(os.path.join(result_path, "Gresult.txt"), 'a')
fo = open(os.path.join(result_path, "Fresult.txt"), 'a')
if w:
  f.write("LENGTH = 12")

for fol in folders[7:8]:
  
  mod = models.Sequential()
  mod.add(layers.Conv2D(16, (5,5), activation='relu', input_shape=(300,300,1)))
  #mod.add(layers.MaxPooling2D((2,2)))
  mod.add(layers.Conv2D(32, (3,3), activation='relu'))
  #mod.add(layers.MaxPooling2D((2,2)))
  mod.add(layers.Flatten())
  mod.add(layers.Dropout(0.2))
  mod.add(layers.Dense(256, activation='relu'))
  mod.add(layers.Dropout(0.4))
  mod.add(layers.Dense(128, activation='relu'))
  mod.add(layers.Dense(1, activation='sigmoid'))
    
  mod.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-3), metrics=['acc'])
  
  #mod.summary()
  
  if(os.path.isdir(os.path.join(genuine_path, fol)) and os.path.isdir(os.path.join(forgery_path, fol))):
    x_genuine, x_test = preProcessImage(os.path.join(genuine_path, fol), length=12)
    x_forgery = preProcessImage(os.path.join(forgery_path, fol), length=3)[1]
    if(os.path.isfile(os.path.join(modelsave_path, fol, fol+'.h5s'))):
      mod.load_weights(os.path.join(modelsave_path, fol, fol+'.h5'))
    else:
      mod.fit(np.concatenate((x_genuine, x_random)).reshape(x_genuine.shape[0]+x_random.shape[0], 300,300,1), np.concatenate((np.full(x_genuine.shape[0],1),np.full(x_random.shape[0],0))), epochs=10, verbose=1, shuffle=True)
      if(os.path.isdir(os.path.join(modelsave_path, fol))==False):
        os.mkdir(os.path.join(modelsave_path, fol))
      #mod.save_weights(os.path.join(modelsave_path, fol, fol+'.h5'))
      
    print('\n'+fol+'\nGenuine: ')
    if w:
      f.write('\n'+fol+'\nGenuine: ')
    for i in x_test:
      predict = mod.predict(np.array([i.reshape(300,300,1),]))
      print('+ ', str(predict*100), '%', end='   ')
      if w:
        f.write('+ '+ str(predict[0][0]*100)+ '%, ')
    
    print('\nForgery: ')
    if w:
      f.write('\nForgery: ')
    for i in x_forgery:
      predict = mod.predict(np.array([i.reshape(300,300,1),]))
      print('- ', str(predict*100), '%', end='   ')
      if w:
        f.write('- '+ str(predict[0][0]*100)+ '%, ')
    print('\n')
    if w:
      f.write('\n')
    
f.close()  


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


100%|██████████| 3/3 [00:05<00:00,  1.83s/it]

Instructions for updating:
Use tf.cast instead.


Epoch 1/10


In [0]:
for i in range(3):
  plt.imshow(x_random[i], cmap='gray')
  plt.grid(False)
  plt.show()

In [0]:
mod = models.Sequential()
mod.add(layers.Conv2D(16, (5,5), activation='relu', input_shape=(300,300,1)))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Conv2D(32, (3,3), activation='relu'))
mod.add(layers.MaxPooling2D((3,3)))
mod.add(layers.Conv2D(32, (3,3), activation='relu'))
mod.add(layers.MaxPooling2D((3,3)))
mod.add(layers.Flatten())
mod.add(layers.Dropout(0.2))
mod.add(layers.Dense(256, activation='relu'))
mod.add(layers.Dropout(0.4))
mod.add(layers.Dense(128, activation='relu'))
mod.add(layers.Dense(1, activation='sigmoid'))

mod.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-3), metrics=['acc'])

In [0]:
mod = models.Sequential()
mod.add(layers.Conv2D(16, (5,5), activation='relu', input_shape=(300,300,1)))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Conv2D(32, (3,3), activation='relu'))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Flatten())
mod.add(layers.Dropout(0.2))
mod.add(layers.Dense(256, activation='relu'))
mod.add(layers.Dropout(0.4))
mod.add(layers.Dense(128, activation='relu'))
mod.add(layers.Dense(1, activation='sigmoid'))

mod.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-3), metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 296, 296, 16)      416       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 148, 148, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 146, 146, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7200)              0         
__________

In [0]:
from keras.utils import plot_model
plot_model(mod, to_file='model.png', show_shapes=True)


In [0]:
allgen = '/content/drive/My Drive/datasets/SigResize/AllGenuine'
allsave = '/content/drive/My Drive/datasets/SigResize/PreProcessedGenuine'

In [0]:
from shutil import copyfile

In [0]:
i = enumerate(folders[:10])

In [0]:
l = list(i)

In [0]:
l

[(0, '1'),
 (1, '2'),
 (2, '3'),
 (3, '4'),
 (4, '5'),
 (5, '6'),
 (6, '7'),
 (7, '8'),
 (8, '9'),
 (9, '10')]

In [0]:
for fol in folders:
  copyfile(os.path.join(genuine_path, fol, '1.tif'), os.path.join(allgen, fol+'.tif'))

In [0]:
def preProcessImageSave(train_path, save_path, final_img_size = (300,300), length=0, getEnd=False, power_law=False, segment=True, log_transform=False):
  train_batch = os.listdir(train_path)
  x_train = []
  train_data = [x for x in train_batch if x.endswith('png') or x.endswith('PNG') or x.endswith('jpg') or x.endswith('JPG') or x.endswith('tif') or x.endswith('TIF')]
  train_data.sort(key = lambda x: int(x.split('.')[0]))
  
  if(length==0 or (length>len(train_data))):
    length = len(train_data)

  for sample in tqdm(train_data[:length]):
    img_path = os.path.join(train_path, sample)
    #importing images from drive
    #x = image.load_img(img_path)
    #img = image.img_to_array(x)
    img = cv2.imread(img_path)
        
    #Perfom Median blur on image
    mbvalue = int(np.max(img.shape)/200)
    mbvalue = mbvalue if mbvalue%2==1 else mbvalue+1
    img = cv2.medianBlur(img, mbvalue)

    #changing RGB to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    #resize image to 600xH
    img = cv2.resize(img, (600, int(600*float(img.shape[0])/img.shape[1])))
    
    #if power_law enabled
    if(power_law):
      img = img**0.9
      img[img>255]=255
      img[img<0]=0
      img = img.astype('uint8')
          
    #denoising the grayscale image
    img = cv2.fastNlMeansDenoising(img, None, 10, 21)
    
    if (log_transform):
        img = (np.log(img+1)/(np.log(10+np.max(img))))*255
        img=img.astype('uint8')
    
    #Threshold binary image
    avg = np.average(img)
    _,image = cv2.threshold(img, int(avg)-30, 255, cv2.THRESH_BINARY)
            
    #segment the signature section only
    if(segment):
      seg = segmentImage(image)
      image = image[seg[2]:seg[3], seg[0]:seg[1]]
          
    #padding to make image into square
    lp, rp, tp, bp = (0,0,0,0)
    if(image.shape[0]>image.shape[1]):
      lp = int((image.shape[0]-image.shape[1])/2)
      rp = lp
    elif(image.shape[1]>image.shape[0]):
      tp = int((image.shape[1]-image.shape[0])/2)
      bp = tp
    image_padded = cv2.copyMakeBorder(image, tp, bp, lp, rp, cv2.BORDER_CONSTANT, value=255)

    #resizing the image
    img = cv2.resize(image_padded, final_img_size)

    #producing image negative
    img = 255-img

    #skeletonizing image
    #img = thin(img/255)

    img = img.astype('bool')

    plt.imsave(os.path.join(save_path, sample), img, cmap='gray')
    
    #appending it in list
    x_train.append(img)

  #converting it into np-array  
  x_train = np.array(x_train)
  return x_train

In [0]:
imgs = preProcessImageSave(allgen, allsave)

100%|██████████| 115/115 [02:34<00:00,  1.54s/it]


In [0]:
plt.imsave('1.tif',imgs[0], cmap='gray')

In [0]:
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_312 (Conv2D)          (None, 292, 292, 16)      1312      
_________________________________________________________________
max_pooling2d_310 (MaxPoolin (None, 146, 146, 16)      0         
_________________________________________________________________
conv2d_313 (Conv2D)          (None, 142, 142, 32)      12832     
_________________________________________________________________
max_pooling2d_311 (MaxPoolin (None, 71, 71, 32)        0         
_________________________________________________________________
conv2d_314 (Conv2D)          (None, 69, 69, 32)        9248      
_________________________________________________________________
max_pooling2d_312 (MaxPoolin (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_315 (Conv2D)          (None, 22, 22, 16)        2064      
__________

In [0]:
for fol in folders[10:30]:
  print("Train "+fol)
  if (os.path.isfile(os.path.join(preprocessed_path, fol, fol+'.csv'))):
    csvReader(os.path.join(preprocessed_path, fol, fol))
  else:
    img_all = preProcessImageAll(os.path.join(genuine_path, fol))
    if(os.path.isdir(os.path.join(preprocessed_path, fol))==False):
        os.mkdir(os.path.join(preprocessed_path, fol))
    csvWriter(os.path.join(preprocessed_path, fol, fol), img_all)


  0%|          | 0/27 [00:00<?, ?it/s]

Train 11



100%|██████████| 27/27 [00:59<00:00,  2.33s/it]

  0%|          | 0/27 [00:00<?, ?it/s]

Train 12



100%|██████████| 27/27 [01:02<00:00,  2.36s/it]

  0%|          | 0/27 [00:00<?, ?it/s]

...


100%|██████████| 27/27 [01:19<00:00,  2.68s/it]

  0%|          | 0/27 [00:00<?, ?it/s]

Train 29



100%|██████████| 27/27 [00:59<00:00,  2.24s/it]

  0%|          | 0/27 [00:00<?, ?it/s]

Train 30



100%|██████████| 27/27 [01:01<00:00,  2.37s/it]


In [0]:
for fol in folders[0:30]:
  print("Train "+fol)
  if (os.path.isfile(os.path.join(preprocessed_path, fol, fol+'Forgery.csv'))):
    csvReader(os.path.join(preprocessed_path, fol, fol+'Forgery'))
  else:
    img_all = preProcessImageAll(os.path.join(forgery_path, fol))
    if(os.path.isdir(os.path.join(preprocessed_path, fol))==False):
        os.mkdir(os.path.join(preprocessed_path, fol))
    csvWriter(os.path.join(preprocessed_path, fol, fol+'Forgery'), img_all)


  0%|          | 0/3 [00:00<?, ?it/s]

Train 1



100%|██████████| 3/3 [00:03<00:00,  1.22s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

Train 2



100%|██████████| 3/3 [00:03<00:00,  1.22s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

...


100%|██████████| 3/3 [00:06<00:00,  2.20s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

Train 29



100%|██████████| 3/3 [00:03<00:00,  1.25s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

Train 30



100%|██████████| 3/3 [00:03<00:00,  1.25s/it]


In [0]:
f=open(os.path.join(result_path, 'S22Imgresult.txt'), 'w')
g=open(os.path.join(result_path, 'S22Imggen.txt'), 'w')
fo = open(os.path.join(result_path, 'S22Imgfor.txt'),'w')
f.close()  
g.close()
fo.close()


In [0]:
mod = models.Sequential()
mod.add(layers.Conv2D(16, (9,9), activation='relu', input_shape=(300,300,1)))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Conv2D(32, (5,5), activation='relu'))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Conv2D(32, (3,3), activation='relu'))
mod.add(layers.MaxPooling2D((3,3)))
mod.add(layers.Conv2D(16, (2,2), activation='relu'))
mod.add(layers.MaxPooling2D((2,2)))
mod.add(layers.Flatten())
mod.add(layers.Dropout(0.2))
mod.add(layers.Dense(256, activation='relu'))
mod.add(layers.Dropout(0.4))
mod.add(layers.Dense(128, activation='relu'))
mod.add(layers.Dense(1, activation='sigmoid'))

In [0]:
!pip install neutron

     |████████████████████████████████| 3.4MB 22.5MB/s 
     |████████████████████████████████| 143kB 50.7MB/s 
     |████████████████████████████████| 61kB 23.5MB/s 
     |████████████████████████████████| 122kB 41.3MB/s 
     |████████████████████████████████| 1.6MB 50.8MB/s 
     |████████████████████████████████| 133kB 51.0MB/s 
     |████████████████████████████████| 440kB 45.9MB/s 
     |████████████████████████████████| 81kB 34.5MB/s 
     |████████████████████████████████| 225kB 52.4MB/s 
     |████████████████████████████████| 1.0MB 49.0MB/s 
     |████████████████████████████████| 501kB 54.9MB/s 
     |████████████████████████████████| 102kB 39.1MB/s 
     |████████████████████████████████| 102kB 17.3MB/s 
     |████████████████████████████████| 2.1MB 60.1MB/s 
     |████████████████████████████████| 51kB 22.0MB/s 
     |████████████████████████████████| 593kB 46.6MB/s 
     |████████████████████████████████| 112kB 57.8MB/s 
     |████████████████████████████████| 184kB 62.7M

In [0]:
!pip install netron

     |████████████████████████████████| 993kB 28.6MB/s 


In [0]:
import netron

In [0]:
netron.start()